In [14]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [15]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from hntm_prod import HierarchicalNeuralProdTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [16]:
config = get_config(nb_name)

In [17]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [18]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [19]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [20]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [21]:
if 'sess' in globals(): sess.close()
if not config.prod:
    Model = HierarchicalNeuralTopicModel
else:
    Model = HierarchicalNeuralProdTopicModel
model = Model(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [22]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = Model(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000    109   10    9  111.45  492  110.51  0.92  0.02  104.07  464  102.90   
10000   137   20   19  110.94  468  109.64  1.25  0.05  103.52  433  101.74   
15000   120   30   29  110.61  452  109.03  1.50  0.07  103.26  419  101.21   
20000   116   40   39  110.38  441  108.63  1.67  0.08  103.07  411  100.93   
25000   116   50   49  110.22  434  108.35  1.79  0.08  103.02  408  100.81   
30000   146   60   59  110.09  428  108.14  1.87  0.08  102.96  404  100.70   
35000    97   70   69  110.00  424  107.97  1.94  0.08  102.92  401  100.62   
40000    85   80   79  109.91  420  107.83  2.00  0.08  102.86  399  100.55   
45000    97   90   89  109.85  417  107.71  2.05  0.08  102.85  398  100.53   
50000   129  100   99  109.78  415  107.61  2.09  0.08  102.73  395  100.37   
55000   135  110  109  109.73  413  107.52  2.13  0.08  102.73  393  100.34   
60000   101  120  119  109.68  410  107.43  2.16  0.08  102.69  392  100.28   
65000   105  130  129  109.64  409  107.36  2.19  0.08  102.67  390  100.23   
70000   125  140  139  109.60  407  107.29  2.22  0.09  102.67  389  100.14   
75000   116  150  149  109.57  405  107.24  2.25  0.08  102.62  390  100.22   
80000   118  160  159  109.53  404  107.18  2.27  0.08  102.60  388  100.12   
85000   103  170  169  109.50  403  107.13  2.29  0.08  102.61  389  100.19   
90000   125  180  179  109.47  402  107.08  2.32  0.08  102.57  386  100.04   
95000   118  190  189  109.45  401  107.04  2.33  0.07  102.60  388  100.15   
100000  127  200  199  109.43  399  107.00  2.35  0.07  102.54  384   99.99   
105000   81  210  209  109.41  399  106.96  2.37  0.07  102.55  386  100.07   
110000  107  220  219  109.39  398  106.93  2.39  0.07  102.53  383   99.96   
115000  112  230  229  109.37  397  106.90  2.40  0.07  102.53  385  100.03   
120000  145  240  239  109.35  396  106.86  2.41  0.07  102.52  382   99.92   
125000  113  250  249  109.33  395  106.84  2.43  0.07  102.49  383   99.97   

                     TEST:      SPEC:             HIER:        
          KL   REG    LOSS  PPL     1     2     3 CHILD OTHER  
5000    1.16  0.02  102.05  456  0.35  0.41  0.72  0.24  0.13  
10000   1.69  0.08  101.59  428  0.42  0.59  0.73  0.26  0.11  
15000   1.95  0.09  101.29  413  0.49  0.69  0.76  0.27  0.11  
20000   2.05  0.09  101.16  406  0.55  0.70  0.82  0.16  0.05  
25000   2.11  0.09  101.08  402  0.59  0.74  0.77  0.26  0.06  
30000   2.17  0.08  101.04  399  0.62  0.74  0.86  0.10  0.03  
35000   2.21  0.09  101.00  397  0.65  0.74  0.86  0.09  0.02  
40000   2.24  0.07  100.91  394  0.64  0.76  0.87  0.07  0.01  
45000   2.24  0.08  100.90  394  0.65  0.81  0.82  0.15  0.04  
50000   2.30  0.07  100.80  389  0.68  0.82  0.88  0.07  0.01  
55000   2.31  0.08  100.79  389  0.67  0.84  0.86  0.11  0.03  
60000   2.35  0.06  100.76  387  0.66  0.84  0.89  0.09  0.00  
65000   2.37  0.07  100.74  386  0.68  0.84  0.87  0.10  0.03  
70000   2.40  0.13  100.76  384  0.69  0.87  0.90  0.07  0.00  
75000   2.38  0.02  100.76  384  0.70  0.80  0.90  0.09  0.00  
80000   2.42  0.06  100.67  383  0.72  0.84  0.88  0.13  0.01  
85000   2.40  0.02  100.67  383  0.71  0.80  0.90  0.09  0.00  
90000   2.47  0.06  100.63  381  0.73  0.84  0.89  0.13  0.01  
95000   2.43  0.02  100.63  381  0.79  0.80  0.90  0.09  0.00  
100000  2.50  0.06  100.63  380  0.79  0.84  0.90  0.12  0.01  
105000  2.46  0.02  100.63  380  0.79  0.81  0.90  0.09  0.00  
110000  2.51  0.06  100.61  379  0.80  0.84  0.91  0.10  0.01  
115000  2.48  0.02  100.61  379  0.78  0.81  0.90  0.09  0.00  
120000  2.54  0.06  100.57  378  0.80  0.84  0.91  0.10  0.01  
125000  2.51  0.02  100.57  378  0.79  0.81  0.91  0.09  0.00

0 R: 1.000 P: 0.471 wheels straps camera pack lenses gear handle shoulder broke strap
   1 R: 0.255 P: 0.128 dvd usb adapter drive netbook aspire power player wireless acer
     11 R: 0.063 P: 0.063 port usb cable adapter impact flap drive neoprene closure plug
     13 R: 0.065 P: 0.065 & retina ; aspire sleeve display acer smell dimensions inch
   2 R: 0.121 P: 0.061 ! christmas loves gift daughter shipping husband amazon & shipped
     22 R: 0.031 P: 0.031 retina & sleeve ; smell neoprene chemical dimensions width display
     21 R: 0.029 P: 0.029 refund contacted customer service return seller told receive shipped shipping
   4 R: 0.152 P: 0.063 mcover typing cover keyboard ipearl purple install fingerprints protector blue
     41 R: 0.089 P: 0.089 speck cracked cover cracking crack cracks rubberized bottom kuzy apple
{0: [1, 2, 4, 3], 1: [11, 13, 12], 2: [23], 4: [41, 42], 3: [31]}


KeyboardInterrupt: 